In [8]:
from copy import deepcopy
import itertools
import pickle as pkl

import interfere
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np

COLORS = [str(v) for v in mcolors.TABLEAU_COLORS.values()]
plt.rcParams["figure.dpi"] = 150

NOTEBOOK_RNG = np.random.default_rng(101)
SEED = 1
REPS = 50


# All Dynamic Models for Experiment 1: The Effect of Stochastic Noise

The following cells contain the parameters of the models that will be used to
study the impact of stochastic noise on the ability of inference methods to
reason about interventions.

The following cell contains all packages and versions used to generate parameter
settings:

```
appnope==0.1.3
asttokens==2.4.1
cmake==3.28.1
comm==0.2.1
contourpy==1.2.0
cycler==0.12.1
debugpy==1.8.0
decorator==5.1.1
derivative==0.6.0
exceptiongroup==1.2.0
executing==2.0.1
filelock==3.13.1
fonttools==4.47.2
fsspec==2023.12.2
importlib-metadata==7.0.1
importlib-resources==6.1.1
iniconfig==2.0.0
-e git+https://github.com/djpasseyjr/interfere.git@5a31f8272698e081253f24d6b7a65a8a6a58984a#egg=interfere
ipykernel==6.29.0
ipython==8.18.1
jedi==0.19.1
Jinja2==3.1.3
joblib==1.3.2
jupyter_client==8.6.0
jupyter_core==5.7.1
kiwisolver==1.4.5
MarkupSafe==2.1.3
matplotlib==3.8.2
matplotlib-inline==0.1.6
mpmath==1.3.0
nest-asyncio==1.6.0
networkx==3.2.1
numpy==1.26.3
packaging==23.2
pandas==2.1.4
parso==0.8.3
patsy==0.5.6
pexpect==4.9.0
pillow==10.2.0
platformdirs==4.1.0
pluggy==1.3.0
prompt-toolkit==3.0.43
psutil==5.9.8
ptyprocess==0.7.0
pure-eval==0.2.2
pyclustering @ git+https://github.com/djpasseyjr/pyclustering@6bb2311e318d3b6ae7ef88f88ed19ec25e959b7f
Pygments==2.17.2
pyparsing==3.1.1
pysindy==1.7.5
pytest==7.4.4
python-dateutil==2.8.2
pytz==2023.3.post1
pyzmq==25.1.2
scikit-base==0.6.2
scikit-learn==1.3.2
scipy==1.11.4
scs==3.2.4.post1
sdeint==0.3.0
six==1.16.0
sktime==0.25.0
stack-data==0.6.3
statsmodels==0.14.1
sympy==1.12
threadpoolctl==3.2.0
tomli==2.0.1
torch==2.1.2
tornado==6.4
traitlets==5.14.1
typing_extensions==4.9.0
tzdata==2023.4
wcwidth==0.2.13
zipp==3.17.0
```

In [9]:
all_params = [] 

In [10]:
def plot_helper(params, noise_params):
    """Plots the model and the change in response to intervention.
    """
    return None
    X, X_do = interfere.generate_counterfactual_forecasts(**params)
    
    X_noise, X_do_noise = interfere.generate_counterfactual_forecasts(
        **noise_params)
    
    t = params["time_points"]

    train_idx, dims = X_do.shape
    fig, ax = plt.subplots(dims, 2, figsize=(7, 10))
    for i in range(dims):
        ax[i, 0].plot(t, X[:, i], alpha=0.7, c=COLORS[0])
        ax[i, 0].plot(t[-train_idx:], X_do[:, i], alpha=0.7, c=COLORS[3])

    for i in range(dims):
        ax[i, 1].plot(t, X_noise[:, i], alpha=0.7, c=COLORS[0])
        ax[i, 1].plot(t[-train_idx:], X_do_noise[:, i], alpha=0.7, c=COLORS[3])

    ax[-1, -1].plot([0, 0], [0, 0], alpha=0.7, c=COLORS[0], label="Observed Signals")
    ax[-1, -1].plot([0, 0], [0, 0], alpha=0.7, c=COLORS[3], label="Response to Intervention")

    plt.legend(loc=(0.2, -.5))
    plt.suptitle(params["model_type"].__name__ + "\n\nMeasurement noise (left) v.s. Stocastic and measurement noise (right)")
    plt.tight_layout()
    plt.show()

### Define Time Scales

In [11]:
def uniform_init_cond(min_x, max_x, dim, reps, rng=NOTEBOOK_RNG):
    """Generates initial conditions for dynamic models."""
    return [
        (max_x - min_x) * rng.random(dim) + min_x for i in range(reps)
    ]

### Arithmetic Brownian Motion

In [12]:
dim = 10

params = dict(
    model_type=interfere.dynamics.ArithmeticBrownianMotion,
    model_params={
        "mu": np.array([0.1, -0.3, 0.06, -0.01, -0.2,
                        -0.05, 0.15, 0.22, -0.17, -0.01]), 
        "sigma": np.zeros(dim),
        "measurement_noise_std": 0.3 * np.ones(dim)
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 4.0},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = np.ones(dim)

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Attracting Fixed Point 4D

In [13]:
dim = 4

params = dict(
    model_type=interfere.dynamics.attracting_fixed_point_4d_linear_sde,
    model_params={"sigma": 0.0, "measurement_noise_std": 0.3 * np.ones(4)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 4.0},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Belozyorov 3D Quadratic SDE

In [14]:
dim = 3

params = dict(
    model_type=interfere.dynamics.Belozyorov3DQuad,
    model_params={
        "mu": 1.81,
        "sigma": 0.0,
        "measurement_noise_std": 0.1 * np.ones(3)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 5.0},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [15]:
dim = 3

params = dict(
    model_type=interfere.dynamics.Belozyorov3DQuad,
    model_params={
        "mu": 1.4,
        "sigma": 0.0,
        "measurement_noise_std": 0.1 * np.ones(3)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 5.0},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [16]:
dim = 3

params = dict(
    model_type=interfere.dynamics.Belozyorov3DQuad,
    model_params={
        "mu": 2.2,
        "sigma": 0.0,
        "measurement_noise_std": 0.1 * np.ones(3)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 5.0},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Coupled logistic maps

In [17]:
dim = 10

params = dict(
    model_type=interfere.dynamics.coupled_logistic_map,
    model_params={
        "adjacency_matrix": np.ones((dim, dim)),
        "eps": 0.5,
        "logistic_param": 3.72,
        "sigma": 0.0,
        "measurement_noise_std": 0.01 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 1.0},
    initial_conds=uniform_init_cond(0, 1, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [18]:
dim = 10

params = dict(
    model_type=interfere.dynamics.coupled_logistic_map,
    model_params={
        # Two cycles and isolated node
        "adjacency_matrix": np.array([
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ]),
        "eps": 0.9,
        "logistic_param": 3.72,
        "sigma": 0.0,
        "measurement_noise_std": 0.01 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 1.0},
    initial_conds=uniform_init_cond(0, 1, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [19]:
dim = 10

params = dict(
    model_type=interfere.dynamics.coupled_logistic_map,
    model_params={
        # Two cycles and isolated node
        "adjacency_matrix": np.array([
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ]),
        "eps": 0.3,
        "logistic_param": 3.0,
        "sigma": 0.0,
        "measurement_noise_std": 0.01 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 1.0},
    initial_conds=uniform_init_cond(0, 1, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Coupled Lattice Map (quadratic map)

In [20]:
dim = 10

params = dict(
    model_type=interfere.dynamics.coupled_map_1dlattice_chaotic_brownian,
    model_params={
        "dim": dim,
        "sigma": 0.0,
        "measurement_noise_std": 0.05 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    initial_conds=uniform_init_cond(0, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [21]:
dim = 10

params = dict(
    model_type=interfere.dynamics.coupled_map_1dlattice_chaotic_traveling_wave,
    model_params={
        "dim": dim,
        "sigma": 0.0,
        "measurement_noise_std": 0.05 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 0.5},
    initial_conds=uniform_init_cond(0, 1, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [22]:
dim = 10

params = dict(
    model_type=interfere.dynamics.coupled_map_1dlattice_defect_turbulence,
    model_params={
        "dim": dim,
        "sigma": 0.0,
        "measurement_noise_std": 0.01 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    initial_conds=uniform_init_cond(0, 1, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [23]:
dim = 10

params = dict(
    model_type=interfere.dynamics.coupled_map_1dlattice_frozen_chaos,
    model_params={
        "dim": dim,
        "sigma": 0.0,
        "measurement_noise_std": 0.02 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    initial_conds=uniform_init_cond(0, 1, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [24]:
dim = 10

params = dict(
    model_type=interfere.dynamics.coupled_map_1dlattice_pattern_selection,
    model_params={
        "dim": dim,
        "sigma": 0.0,
        "measurement_noise_std": 0.02 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    initial_conds=uniform_init_cond(0, 1, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [25]:
dim = 10

params = dict(
    model_type=interfere.dynamics.coupled_map_1dlattice_spatiotemp_chaos,
    model_params={
        "dim": dim,
        "sigma": 0.0,
        "measurement_noise_std": 0.05 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    initial_conds=uniform_init_cond(0, 1, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [26]:
dim = 10

params = dict(
    model_type=interfere.dynamics.coupled_map_1dlattice_spatiotemp_intermit1,
    model_params={
        "dim": dim,
        "sigma": 0.0,
        "measurement_noise_std": 0.05 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 0.5},
    initial_conds=uniform_init_cond(0, 1, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [27]:
dim = 10

params = dict(
    model_type=interfere.dynamics.coupled_map_1dlattice_traveling_wave,
    model_params={
        "dim": dim,
        "sigma": 0.0,
        "measurement_noise_std": 0.05 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 0.5},
    initial_conds=uniform_init_cond(0, 1, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Damped Oscilator

In [28]:
dim = 2

params = dict(
    model_type=interfere.dynamics.DampedOscillator,
    model_params={"m": 1.0, "c": 2, "k": 10, "sigma": 0,
                  "measurement_noise_std": 0.1 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [29]:
dim = 2

params = dict(
    model_type=interfere.dynamics.DampedOscillator,
    model_params={"m": 30.0, "c": 0.5, "k": 2, "sigma": 0,
                  "measurement_noise_std": 0.1 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Geometric Browian Motion

In [30]:
dim = 10

params = dict(
    model_type=interfere.dynamics.GeometricBrownianMotion,
    model_params={
        "mu": np.array([
            -0.2, 0.003, -10, -0.0007, -1.3, -6, -0.8, -7, -0.38, -99.0]),
        "sigma": np.zeros(dim),
        "measurement_noise_std": 0.01 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 0.5},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.05 * np.ones(dim)

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [31]:
dim = 10

params = dict(
    model_type=interfere.dynamics.GeometricBrownianMotion,
    model_params={
        "mu": np.array([
            -0.2, 0.003, -10, -0.0007, -1.3, -6, -0.8, -7, -0.38, -99.0]),
        "sigma": np.zeros(dim),
        "measurement_noise_std": 0.01 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 0.5},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 3 * np.ones(dim)

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Hodgkin Huxley 

In [32]:
from pyclustering.nnet.hhn import hhn_parameters

In [33]:
dim = 10

hh_ps = hhn_parameters()
hh_ps.w1 = 0
hh_ps.w2 = 1.0
hh_ps.w3 = 0


params = dict(
    model_type=interfere.dynamics.HodgkinHuxleyPyclustering,
    model_params={
        "stimulus": NOTEBOOK_RNG.random(dim) * 50 - 20,
        "sigma": np.zeros(dim),
        "measurement_noise_std": 5 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -20},
    initial_conds=uniform_init_cond(-40, 10, dim, REPS),
    start_time=0, end_time=200, dt=0.0001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 5 * np.ones(dim)

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Imaginary Roots 4D

In [34]:
dim = 4

params = dict(
    model_type=interfere.dynamics.imag_roots_4d_linear_sde,
    model_params={"sigma": 0,
                  "measurement_noise_std": np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    initial_conds=uniform_init_cond(0, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Kuramoto Oscilators

In [35]:
dim = 10

params = dict(
    model_type=interfere.dynamics.Kuramoto,
    model_params={
        "omega": NOTEBOOK_RNG.random(10) * np.pi / 2,
        "K": 2.5,
        # A chain network
        "adjacency_matrix": np.array([
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
        ]),
        "sigma": 0,
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [36]:
dim = 10

params = dict(
    model_type=interfere.dynamics.Kuramoto,
    model_params={
        "omega": NOTEBOOK_RNG.random(10) * np.pi / 2,
        "K": 2.5,
        # A cycle and isolated node
        "adjacency_matrix": np.array([
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ]),
        "sigma": 0,
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [37]:
dim = 10

params = dict(
    model_type=interfere.dynamics.Kuramoto,
    model_params={
        # All nodes have the same fundamental frequency
        "omega": np.ones(dim),
        "K": 2.5,
        # A cycle and isolated node
        "adjacency_matrix": np.array([
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ]),
        "sigma": 0,
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    # Nodes begin synchronized but the intervention reveals coupling.
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.1

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Kuramoto-Sakaguchi

In [38]:
dim = 10

params = dict(
    model_type=interfere.dynamics.KuramotoSakaguchi,
    model_params={
        "omega": NOTEBOOK_RNG.random(dim) * np.pi / 2,
        "K": 2.5,
        # A chain network
        "adjacency_matrix": np.array([
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
        ]),
        "phase_frustration": np.array([
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
        ]),
        "sigma": 0,
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [39]:
dim = 10

params = dict(
    model_type=interfere.dynamics.KuramotoSakaguchi,
    model_params={
        "omega": NOTEBOOK_RNG.random(dim) * np.pi / 2,
        "K": 2.5,
        # Three cycle and isolated node
        # A cycle and isolated node
        "adjacency_matrix": np.array([
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ]),
        "phase_frustration": np.array([
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ]),
        "sigma": 0,
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    initial_conds=uniform_init_cond(-1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [40]:
dim = 10

params = dict(
    model_type=interfere.dynamics.KuramotoSakaguchi,
    model_params={
        # All nodes have the same fundamental frequency
        "omega": np.ones(dim),
        "K": 2.5,
        # A cycle and isolated node
        "adjacency_matrix": np.array([
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ]),
        "phase_frustration": np.array([
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ]),
        "sigma": 0,
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.5},
    # Nodes begin synchronized but the intervention reveals coupling.
    initial_conds=uniform_init_cond(1, 1, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Liping Finance Model

In [41]:
dim = 3

params = dict(
    model_type=interfere.dynamics.Liping3DQuadFinance,
    model_params={
        "sigma": 0,
        "measurement_noise_std": 0.25 * np.ones(dim),
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": -0.1},
    initial_conds=uniform_init_cond(-2, 2, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.5

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Lotka Voltera

In [42]:
dim = 10

params = dict(
    model_type=interfere.dynamics.LotkaVolteraSDE,
    model_params={
        "growth_rates": 5 * NOTEBOOK_RNG.random(dim),
        "capacities": 20 * np.ones(dim),
        # A cycle and isolated node
        "interaction_mat": np.array([
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ]),
        "sigma": 0,
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 5.0},
    initial_conds=uniform_init_cond(0, 10, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.1

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [43]:
dim = 10

params = dict(
    model_type=interfere.dynamics.LotkaVolteraSDE,
    model_params={
        # All nodes have the same growth rate
        "growth_rates": np.ones(dim),
        "capacities": 20 * np.ones(dim),
        # Three cycle and isolated node
        "interaction_mat": np.array([
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ]),
        "sigma": 0,
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 5.0},
    # Nodes begin synchronized but the intervention reveals coupling.
    initial_conds=uniform_init_cond(2, 2, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.1

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [44]:
dim = 10

params = dict(
    model_type=interfere.dynamics.LotkaVolteraSDE,
    model_params={
        # All nodes have the same growth rate
        "growth_rates": np.ones(dim),
        "capacities": 20 * np.ones(dim),
        # Erdos-Renyi with mean degree 3
        "interaction_mat": NOTEBOOK_RNG.random((dim, dim)) < 3 / dim,
        "sigma": 0,
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 10.0},
    # Nodes begin synchronized but the intervention reveals coupling.
    initial_conds=uniform_init_cond(0, 6, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.1

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [45]:
dim = 10

params = dict(
    model_type=interfere.dynamics.LotkaVolteraSDE,
    model_params={
        "growth_rates": 2 * NOTEBOOK_RNG.random(dim),
        "capacities": 20 * np.ones(dim),
        # Erdos-Renyi with mean degree 3
        "interaction_mat": NOTEBOOK_RNG.random((dim, dim)) < 3 / dim,
        "sigma": 0,
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 10.0},
    initial_conds=uniform_init_cond(0, 6, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 0.1

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Ornstein Uhlenbeck

In [46]:
dim = 10

# Random orthonormal matrix
U = np.linalg.svd(NOTEBOOK_RNG.random((dim, dim)))[0]
# Random maxtix with all positive eigs
theta = U @ np.diag(10 * NOTEBOOK_RNG.random(dim)) @ U.T

params = dict(
    model_type=interfere.dynamics.OrnsteinUhlenbeck,
    model_params={
        "mu": 0.25 * np.ones(dim),
        # Chain
        "theta": theta,
        "Sigma": np.zeros((dim, dim)),
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 2.0},
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["Sigma"] = 0.1 * NOTEBOOK_RNG.random((dim, dim)) - 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [47]:
dim = 10

# Random orthonormal matrix
U = np.linalg.svd(NOTEBOOK_RNG.random((dim, dim)))[0]
# Random maxtix with all positive eigs
theta = U @ np.diag(0.5 * NOTEBOOK_RNG.random(dim)) @ U.T

params = dict(
    model_type=interfere.dynamics.OrnsteinUhlenbeck,
    model_params={
        "mu": 0.25 * np.ones(dim),
        # Chain
        "theta": theta,
        "Sigma": np.zeros((dim, dim)),
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 2.0},
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["Sigma"] = 0.1 * NOTEBOOK_RNG.random((dim, dim)) - 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [48]:
dim = 10

# Random orthonormal matrix
U = np.linalg.svd(NOTEBOOK_RNG.random((dim, dim)))[0]
# Random maxtix with all positive eigs
theta = U @ np.diag(10 * NOTEBOOK_RNG.random(dim)) @ U.T

params = dict(
    model_type=interfere.dynamics.OrnsteinUhlenbeck,
    model_params={
        "mu": 0.25 * np.ones(dim),
        # Chain
        "theta": theta,
        "Sigma": np.zeros((dim, dim)),
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 2.0},
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["Sigma"] = 0.1 * NOTEBOOK_RNG.random((dim, dim)) - 0.05

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [49]:
dim = 10

# Random orthonormal matrix
U = np.linalg.svd(NOTEBOOK_RNG.random((dim, dim)))[0]
# Random maxtix with all positive eigs
theta = U @ np.diag(10 * NOTEBOOK_RNG.random(dim)) @ U.T

params = dict(
    model_type=interfere.dynamics.OrnsteinUhlenbeck,
    model_params={
        "mu": 0.25 * np.ones(dim),
        # Chain
        "theta": theta,
        "Sigma": np.zeros((dim, dim)),
        "measurement_noise_std": 0.25 * np.ones(dim)},
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 2.0},
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["Sigma"] = 2 * NOTEBOOK_RNG.random((dim, dim)) - 1

plot_helper(params, noise_params)

all_params += [params, noise_params]

### Uncorrelated Noise

In [50]:
dim = 10

params = dict(
    model_type=interfere.dynamics.StandardCauchyNoise,
    model_params={
        "dim": dim,
        "measurement_noise_std": None,
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 0.5},
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [51]:
dim = 10

params = dict(
    model_type=interfere.dynamics.StandardExponentialNoise,
    model_params={
        "dim": dim,
        "measurement_noise_std": None,
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 0.5},
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [52]:
dim = 10

params = dict(
    model_type=interfere.dynamics.StandardGammaNoise,
    model_params={
        "dim": dim,
        "measurement_noise_std": None,
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 0.5},
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [53]:
dim = 10

params = dict(
    model_type=interfere.dynamics.StandardNormalNoise,
    model_params={
        "dim": dim,
        "measurement_noise_std": None,
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 0.5},
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [54]:
dim = 10

params = dict(
    model_type=interfere.dynamics.StandardTNoise,
    model_params={
        "dim": dim,
        "measurement_noise_std": None,
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 0.5},
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=200, dt=0.001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)

plot_helper(params, noise_params)

all_params += [params, noise_params]

### VARMA Models

In [55]:
from statsmodels.tsa.statespace.varmax import VARMAX

def extract_covariance(statsmodels_var_result):
    cov_table = np.array(statsmodels_var_result.summary().tables[-1])
    # Extract entries
    X = np.vectorize(lambda x: x.data)(cov_table)

    # Initialize empty cov matrix
    cov = [["0"]* 10 for i in range(10)]
    for i in range(0, 10):
        for j in range(i+1):
            cov[i][j] = X[j + i + 1, 1]

    cov = np.array(cov, float)
    cov = cov + np.tril(cov).T
    return cov

In [56]:
dim = 10

model = interfere.dynamics.coupled_map_1dlattice_spatiotemp_chaos(**{
    "dim": dim,
    "sigma": 0.0,
    "measurement_noise_std": 0.05 * np.ones(dim)
})

X = model.simulate(np.ones(dim), np.arange(100))
varma = VARMAX(X, order=(2, 3)).fit()

params = dict(
    model_type=interfere.dynamics.VARMA_Dynamics,
    model_params={
        "phi_matrices": varma.coefficient_matrices_var,
        "theta_matrices": varma.coefficient_matrices_vma,
        "sigma": np.zeros((dim, dim)),
        "measurement_noise_std": 0.2 * np.ones(dim),
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 1.5},
    # Nodes begin synchronized.
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = extract_covariance(varma)

plot_helper(params, noise_params)

all_params += [params, noise_params]

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          565     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -1.20567D+01    |proj g|=  1.36165D+01


 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  565      1      3      1     0     0   1.362D+01  -1.206D+01
  F =  -12.056673108402826     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [58]:
dim = 10

model = interfere.dynamics.coupled_map_1dlattice_chaotic_brownian(**{
    "dim": dim,
    "sigma": 0.0,
    "measurement_noise_std": 0.05 * np.ones(dim)
})

X = model.simulate(np.ones(dim), np.arange(100))
varma = VARMAX(X, order=(2, 3)).fit()

params = dict(
    model_type=interfere.dynamics.VARMA_Dynamics,
    model_params={
        "phi_matrices": varma.coefficient_matrices_var,
        "theta_matrices": varma.coefficient_matrices_vma,
        "sigma": np.zeros((dim, dim)),
        "measurement_noise_std": 0.2 * np.ones(dim),
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 1.5},
    # Nodes begin synchronized.
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = extract_covariance(varma)

plot_helper(params, noise_params)

all_params += [params, noise_params]

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          565     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -7.12724D+00    |proj g|=  1.11331D+02

At iterate    5    f= -1.02080D+01    |proj g|=  4.18854D+01


In [ ]:
dim = 10

# Fit a VARMA model to one of the previous dynamical systems
model = interfere.dynamics.LotkaVolteraSDE(
    **{
        # All nodes have the same growth rate
        "growth_rates": np.ones(dim),
        "capacities": 20 * np.ones(dim),
        # Three cycle and isolated node
        "interaction_mat": np.array([
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ]),
        "sigma": 0.01,
        "measurement_noise_std": 0.25 * np.ones(dim)
    }
)

X = model.simulate(10 * np.ones(dim), np.linspace(0, 10, 1000))
varma = VARMAX(X, order=(2, 3)).fit()

params = dict(
    model_type=interfere.dynamics.VARMA_Dynamics,
    model_params={
        "phi_matrices": varma.coefficient_matrices_var,
        "theta_matrices": varma.coefficient_matrices_vma,
        "sigma": np.zeros((dim, dim)),
        "measurement_noise_std": 0.2 * np.ones(dim),
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 1.5},
    # Nodes begin synchronized.
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = extract_covariance(varma)

plot_helper(params, noise_params)

all_params += [params, noise_params]

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          565     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.14585D-01    |proj g|=  2.47477D-01


 This problem is unconstrained.



At iterate    5    f=  5.13944D-01    |proj g|=  1.19200D+00

At iterate   10    f=  5.09452D-01    |proj g|=  1.88325D-01

At iterate   15    f=  5.09087D-01    |proj g|=  5.71470D-01

At iterate   20    f=  5.05938D-01    |proj g|=  4.48452D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  565     22     28      1     0     0   2.596D-01   5.059D-01
  F =  0.50590291980708646     

STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT        


In [ ]:
dim = 10

# Fit a VARMA model to one of the previous dynamical systems
model = interfere.dynamics.LotkaVolteraSDE(
    **{
        # All nodes have the same growth rate
        "growth_rates": NOTEBOOK_RNG.random(dim),
        "capacities": 20 * np.ones(dim),
        # Three cycle and isolated node
        "interaction_mat": np.array([
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
            [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        ]),
        "sigma": 0.01,
        "measurement_noise_std": 0.25 * np.ones(dim)
    }
)

X = model.simulate(10 * np.ones(dim), np.linspace(0, 10, 1000))
varma = VARMAX(X, order=(2, 3)).fit()

params = dict(
    model_type=interfere.dynamics.VARMA_Dynamics,
    model_params={
        "phi_matrices": varma.coefficient_matrices_var,
        "theta_matrices": varma.coefficient_matrices_vma,
        "sigma": np.zeros((dim, dim)),
        "measurement_noise_std": 0.2 * np.ones(dim),
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 1.5},
    # Nodes begin synchronized.
    initial_conds=uniform_init_cond(-3, 3, dim, REPS),
    start_time=0, end_time=2000, dt=1,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = extract_covariance(varma)

plot_helper(params, noise_params)

all_params += [params, noise_params]

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          565     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.18607D-01    |proj g|=  2.83155D-01

At iterate    5    f=  5.17348D-01    |proj g|=  2.43013D+00

At iterate   10    f=  5.09573D-01    |proj g|=  6.39188D-01

At iterate   15    f=  5.09297D-01    |proj g|=  7.31597D-01

At iterate   20    f=  5.03368D-01    |proj g|=  2.52027D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  565     23     28      1     0     0   2.448D-01   5.026D-01
  F =  0.50259722848063548     

STOP: T

### Additional Hodgkin Huxley Models

In [ ]:
dim = 10

stimulus = NOTEBOOK_RNG.random(dim)
params = hhn_parameters()
params.w1 = 0.5
params.w2 = 0
params.w3 = 0
params.deltah = 400


params = dict(
    model_type=interfere.dynamics.HodgkinHuxleyPyclustering,
    model_params={
        "stimulus": NOTEBOOK_RNG.random(dim) * 50 + 20,
        "sigma": np.zeros(dim),
        "measurement_noise_std": 5 * np.ones(dim),
        "parameters": params,
        "type_conn": "list_bdir"
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 70},
    initial_conds=uniform_init_cond(-40, 10, dim, REPS),
    start_time=0, end_time=200, dt=0.0001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 5 * np.ones(dim)

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [ ]:
dim = 10

stimulus = NOTEBOOK_RNG.random(dim)
params = hhn_parameters()
params.w1 = 0.5
params.w2 = 0
params.w3 = 0
params.deltah = 400


params = dict(
    model_type=interfere.dynamics.HodgkinHuxleyPyclustering,
    model_params={
        "stimulus": NOTEBOOK_RNG.random(dim) * 50 + 20,
        "sigma": np.zeros(dim),
        "measurement_noise_std": 5 * np.ones(dim),
        "parameters": params,
        "type_conn": "grid_four"
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 70},
    initial_conds=uniform_init_cond(-40, 10, dim, REPS),
    start_time=0, end_time=200, dt=0.0001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 5 * np.ones(dim)

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [ ]:
dim = 10

stimulus = NOTEBOOK_RNG.random(dim)
params = hhn_parameters()
params.w1 = 0.1
params.w2 = 0
params.w3 = 0.2
params.deltah = 400


params = dict(
    model_type=interfere.dynamics.HodgkinHuxleyPyclustering,
    model_params={
        "stimulus": 40 * np.ones(dim),
        "sigma": np.zeros(dim),
        "measurement_noise_std": 5 * np.ones(dim),
        "parameters": params,
        "type_conn": "grid_four"
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 70},
    initial_conds=uniform_init_cond(0, 0, dim, REPS),
    start_time=0, end_time=200, dt=0.0001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 5 * np.ones(dim)

plot_helper(params, noise_params)

all_params += [params, noise_params]

In [ ]:
dim = 10

stimulus = NOTEBOOK_RNG.random(dim)
params = hhn_parameters()
params.w1 = 0.0
params.w2 = 0.0
params.w3 = 0.0
params.deltah = 400


params = dict(
    model_type=interfere.dynamics.HodgkinHuxleyPyclustering,
    model_params={
        "stimulus": 20 * NOTEBOOK_RNG.random(dim) + 30,
        "sigma": np.zeros(dim),
        "measurement_noise_std": 5 * np.ones(dim),
        "parameters": params,
        "type_conn": "grid_four"
    },
    intervention_type=interfere.PerfectIntervention,
    intervention_params={"intervened_idxs": 0, "constants": 70},
    initial_conds=uniform_init_cond(-70, 30, dim, REPS),
    start_time=0, end_time=200, dt=0.0001,
    rng = np.random.default_rng(SEED)
)

noise_params = deepcopy(params)
noise_params["model_params"]["sigma"] = 5 * np.ones(dim)

plot_helper(params, noise_params)

all_params += [params, noise_params]

## Add different length simulation times to the parameter sets

In [ ]:
dynamics_kwargs = []
dynamics_kwargs_small = []

DISCRETE_MED = 500
DISCRETE_SHORT = 100

CONT_MED = 50
CONT_SHORT = 10

for x in all_params:
    # Make medium and short timescale versions of the parameter set.
    med_x = deepcopy(x)
    short_x = deepcopy(x)

    # Discrete time end times.
    if x["dt"] == 1:
        med_x["end_time"] = DISCRETE_MED
        short_x["end_time"] = DISCRETE_SHORT

    # Continuous time end times.
    else:
        med_x["end_time"] = CONT_MED
        short_x["end_time"] = CONT_SHORT

    dynamics_kwargs += [x, med_x, short_x]
    dynamics_kwargs_small += [short_x]

# Grab five random models
dynamics_kwargs_small = [dynamics_kwargs_small[i] for i in [10, 20, 30, 41, 51]]


## Method Hyper Parameters

In [ ]:
import pysindy as ps 

methods_kwargs = [

    dict(
        method_type=interfere.methods.VAR,
        method_params={},
        method_param_grid={
            "maxlags": [1, 2, 3, 10],
            "trend" : ["c", "ct", "ctt", "n"]
        },
        num_intervention_sims=3,
    ),

    dict(
        method_type=interfere.methods.SINDY,
        method_params={
            "differentiation_method": ps.SINDyDerivative(kind='spectral')
        },
        method_param_grid={
            "optimizer__threshold": [1e-8, 0.0001, 0.001, 0.01, 0.05, 0.1],
            "optimizer__alpha": [0.0, 1e-8, 1e-4, 1e-2, 0.1, 1.0, 5],
            "differentiation_method__kwargs": [
                {'kind': 'finite_difference', 'k': 1},
                {'kind': 'spline', 's': 1e-2},
                {'kind': 'spline', 's': 1e-1},
                {'kind': 'savitzky_golay', 'order': 2, 'left': .1, 'right': .1},
                {'kind': 'savitzky_golay', 'order': 3, 'left': .1, 'right': .1},
                {'kind': 'trend_filtered', 'order': 0, 'alpha': 1e-2}
            ]
        },
        num_intervention_sims=1,
    ),


    dict(
        method_type=interfere.methods.LTSFLinearForecaster,
        method_params={"seq_len": 1, "pred_len": 1},
        method_param_grid={
            "seq_len": [1, 2, 5, 10, 50],
            "lr": [0.001, 0.03, 0.1],
            "pred_len": [1, 2, 3]
        },
        num_intervention_sims=1,

    )

]

# Make a smaller dictionary of methods for testing that everything runs.
# Use fewer hyper parameters.

methods_kwargs_small = [

    dict(
        method_type=interfere.methods.VAR,
        method_params={},
        method_param_grid={
            "maxlags": [1],
            "trend" : ["c"]
        },
        num_intervention_sims=3,

    ),

    dict(
        method_type=interfere.methods.SINDY,
        method_params={
            "differentiation_method": ps.SINDyDerivative(kind='spectral')
        },
        method_param_grid={
            "optimizer__threshold": [1e-8],
            "optimizer__alpha": [0.0],
            "differentiation_method__kwargs": [
                {'kind': 'trend_filtered', 'order': 0, 'alpha': 1e-2}
            ]
        },
        num_intervention_sims=3,
    ),


    dict(
        method_type=interfere.methods.LTSFLinearForecaster,
        method_params={"seq_len": 1, "pred_len": 1},
        method_param_grid={
            "seq_len": [1],
            "lr": [0.03],
            "pred_len": [3]
        },
        num_intervention_sims=3,
    )

]

## Save the parameter sets

In [ ]:
# Full set of dynamic models.
with open("/Users/djpassey/Code/interfere/experiments/exp1_dynamic_models.pkl", "wb") as f:
    pkl.dump(dynamics_kwargs, f)


# Dynamic models with shortest time scale only.
with open("/Users/djpassey/Code/interfere/experiments/exp1_dynamic_models_small.pkl", "wb") as f:
    pkl.dump(dynamics_kwargs_small, f)

# Methods will full hyper parameter options.
with open("/Users/djpassey/Code/interfere/experiments/exp1_inference_methods.pkl", "wb") as f:
    pkl.dump(methods_kwargs, f)

# Methods with less hyper parameters
with open("/Users/djpassey/Code/interfere/experiments/exp1_inference_methods_small.pkl", "wb") as f:
    pkl.dump(methods_kwargs_small, f)
    

In [ ]:
import sys
from sys import getsizeof, stderr
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)

total_size(noise_exp)

NameError: name 'noise_exp' is not defined